In [2]:
from author_style.utils import *
from author_style.preprocessing import *

In [3]:
clean_texts()

In [4]:
df = csv_to_dataframes(output='p')
df

,text,author,title,book_date
0,Pendant que Nous franchissions la porte du Nor...,GUTH Paul,Si j_étais le Bon Dieu,1987
1,"« En 486 après Jésus-Christ, les troupes de Sy...",GUTH Paul,Si j_étais le Bon Dieu,1987
2,Van Eyck présenta La Vierge au chanoine Van de...,GUTH Paul,Si j_étais le Bon Dieu,1987
3,"Un an après l’insolence du soldat, Clovis rass...",GUTH Paul,Si j_étais le Bon Dieu,1987
4,Les hommes se font une idée grotesque du temps...,GUTH Paul,Si j_étais le Bon Dieu,1987
...,...,...,...,...
18773,C’est à bord d’un train de la Southern Pacific...,ECHENOZ Jean,Ravel,2006
18774,"Quelle que soit, pour signer, la solution adop...",ECHENOZ Jean,Ravel,2006
18775,Reste la possibilité d’aller faire un tour dan...,ECHENOZ Jean,Ravel,2006
18776,"Le lendemain matin, il se lève tard, traînant ...",ECHENOZ Jean,Ravel,2006


In [5]:
from author_style.pipeline import *

Index(['text', 'author', 'title', 'book_date', 'preprocess_data', 'word_ratio',
       'unique_word_ratio', 'sentences_ratio', 'stopwords_ratio',
       'vocab richness'],
      dtype='object')


KeyError: "['unique_word'] not in index"

In [6]:
df = csv_to_dataframes(output='p')
df

,text,author,title,book_date
0,Pendant que Nous franchissions la porte du Nor...,GUTH Paul,Si j_étais le Bon Dieu,1987
1,"« En 486 après Jésus-Christ, les troupes de Sy...",GUTH Paul,Si j_étais le Bon Dieu,1987
2,Van Eyck présenta La Vierge au chanoine Van de...,GUTH Paul,Si j_étais le Bon Dieu,1987
3,"Un an après l’insolence du soldat, Clovis rass...",GUTH Paul,Si j_étais le Bon Dieu,1987
4,Les hommes se font une idée grotesque du temps...,GUTH Paul,Si j_étais le Bon Dieu,1987
...,...,...,...,...
18773,C’est à bord d’un train de la Southern Pacific...,ECHENOZ Jean,Ravel,2006
18774,"Quelle que soit, pour signer, la solution adop...",ECHENOZ Jean,Ravel,2006
18775,Reste la possibilité d’aller faire un tour dan...,ECHENOZ Jean,Ravel,2006
18776,"Le lendemain matin, il se lève tard, traînant ...",ECHENOZ Jean,Ravel,2006


In [ ]:
df = features(df)
df

In [ ]:
X = df[[
    'preprocess_data'
]]
y = df["author"]

In [ ]:
X.head(3)

In [ ]:
y.head(2)

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(y)
y_cat = le.transform(y)

In [ ]:
y_cat

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.3, random_state=42)

In [ ]:
import fasttext.util
fasttext.util.download_model('fr', if_exists='ignore')  # English
ft = fasttext.load_model('cc.fr.300.bin')